# 【python できること】 Googleスプレッドシートを使ってデータ連携する方法

### ライブラリ読み込み

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import csv

### 設定

In [2]:
# 設定
json_file = '***.json'
file_name = '***'
sheet_name1 = 'シート1'
sheet_name2 = 'csv_sheet'
csv_file_name = 'Davis.csv'

### スプレッドシートにアクセス

In [3]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# スプレッドシートにアクセス
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
gc = gspread.authorize(credentials)
sh = gc.open(file_name)

### シートの作成と削除

In [4]:
# シートの作成
wks = sh.add_worksheet(title="new worksheet", rows='100', cols='30')

In [5]:
# シートの削除
sh.del_worksheet(wks)

{'spreadsheetId': '1opTtfyXuO2SEeUtqy9UtnMLIozfpXoGXcbieqO3bfvU',
 'replies': [{}]}

### シートリスト取得


In [6]:
sheet_list = [ws.title for ws in sh.worksheets()]
print(sheet_list)

['シート1', 'csv_sheet']


### シートを選択

In [7]:
# シートの選択　シートの番号でも名前でもよい
wks = sh.get_worksheet(0)
wks = sh.worksheet(sheet_name1)

### 書き込み

In [8]:
# 単一セル
wks.update_acell('A1', '1')
wks.update_cell(1, 2, '2')

# 複数セル
list_data = [[1, 2], [3, 4]]
wks.update('A2:B3', list_data)

{'spreadsheetId': '1opTtfyXuO2SEeUtqy9UtnMLIozfpXoGXcbieqO3bfvU',
 'updatedRange': "'シート1'!A2:B3",
 'updatedRows': 2,
 'updatedColumns': 2,
 'updatedCells': 4}

In [9]:
# 書き込むシート作成。すでにあれば読み込む
if sheet_name2 in sheet_list:
    wks = sh.worksheet(sheet_name2)
else:
    wks = sh.add_worksheet(title=sheet_name2, rows='300', cols='10')
# CSVを書き込み
wks.update(list(csv.reader(open(csv_file_name, encoding='cp932'))))

{'spreadsheetId': '1opTtfyXuO2SEeUtqy9UtnMLIozfpXoGXcbieqO3bfvU',
 'updatedRange': 'csv_sheet!A1:F201',
 'updatedRows': 201,
 'updatedColumns': 6,
 'updatedCells': 1206}

In [10]:
# シートデータクリア
sh.values_clear(f"{sheet_name2}!A1:F300")

{'spreadsheetId': '1opTtfyXuO2SEeUtqy9UtnMLIozfpXoGXcbieqO3bfvU',
 'clearedRange': 'csv_sheet!A1:F300'}

In [11]:
# numpy書き込み
import numpy as np
data = np.array([[1,2,3],[1,2,3],[1,2,3]])
wks.update('A1:C3', data.tolist())

{'spreadsheetId': '1opTtfyXuO2SEeUtqy9UtnMLIozfpXoGXcbieqO3bfvU',
 'updatedRange': 'csv_sheet!A1:C3',
 'updatedRows': 3,
 'updatedColumns': 3,
 'updatedCells': 9}

In [12]:
# シートデータクリア
sh.values_clear(f"{sheet_name2}!A1:F300")

{'spreadsheetId': '1opTtfyXuO2SEeUtqy9UtnMLIozfpXoGXcbieqO3bfvU',
 'clearedRange': 'csv_sheet!A1:F300'}

In [13]:
# pandas書き込み
import pandas as pd
df = pd.read_csv(csv_file_name).fillna('')
# 列名とデータを連結して書き込み
wks.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1opTtfyXuO2SEeUtqy9UtnMLIozfpXoGXcbieqO3bfvU',
 'updatedRange': 'csv_sheet!A1:F201',
 'updatedRows': 201,
 'updatedColumns': 6,
 'updatedCells': 1206}

### 読み込み

In [14]:
wks = sh.worksheet(sheet_name1)
# 単一セル
val1 = wks.acell('A1').value
val2 = wks.cell(1, 2).value

print(val1, val2)

# 複数セル
list_data = wks.get('A2:B3')
print(list_data)

1 2
[['1', '2'], ['3', '4']]


In [15]:
# pandas
wks = sh.worksheet(sheet_name2)
df = pd.DataFrame(wks.get_all_records())

In [16]:
# numpy
wks = sh.worksheet(sheet_name2)
array = np.array(wks.get_all_values())